In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Importing the model.
!pip install GitPython
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip

In [ ]:
import git
url="https://github.com/karen-gishyan/Computer-vision-and-Drones-Thesis"
git.Git().clone(url)

In [6]:
import git
import subprocess
import shutil
import os
import sys

class Yolo:

  def __init__(self,working_dir=None):
    self.cwd=working_dir if working_dir != None else "/content"
  
  @staticmethod
  def import_model(cwd):

    url="https://github.com/ultralytics/yolov3"
    repo_name=os.path.basename(url)
    path=os.path.join(cwd,repo_name) 
    print(path)
    
    if  not os.path.exists(path):
      git.Git(cwd).clone(url)
    
    else:    
      user_input=input("Non_Empty directory, replace? press y for Yes, n for No:")
      if user_input=="y":
        shutil.rmtree(path)
        git.Git(cwd).clone(url)
 
    os.chdir("/content/yolov3")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
    os.chdir("/content")
    print("Requrements installed.")
  
  
cwd="/content/"
Yolo().import_model(cwd)

/content/yolov3
Requrements installed.


In [7]:
#Create a txt file with image paths for train and valid.

import os
path_train_images=os.path.normpath("/content/drive/My Drive/Bath Thesis/Augmentation 1/train_images")
path_valid_images=os.path.normpath("/content/drive/My Drive/Bath Thesis/Not Augmented/valid_images")

def txt_with_paths(path_train,path_valid):
      
      print ("Number of Training Images is:",len([file for file  in os.listdir(path_train)]))
      print ("Number of Validation Images is:",len([file for file  in os.listdir(path_valid)]))

      with open(os.path.join("/content","train.txt"),"wt") as train, open(os.path.join("/content","valid.txt"),"wt") as valid:
            
        for train_image in os.listdir(path_train_images):
    
          train_image_path=os.path.join(path_train_images,train_image)
          train.write(train_image_path+"\n")
          
        for valid_image in os.listdir(path_valid_images):
          valid_image_path=os.path.join(path_valid_images,valid_image)        
          valid.write(valid_image_path+ "\n")

txt_with_paths(path_train_images,path_valid_images)

Number of Training Images is: 1400
Number of Validation Images is: 200


In [ ]:
#Creating .data file.

def create_data(path_to_train_txt,path_to_valid_txt,path_to_class_names):
  number_of_classes=str(sum(1 for line in open(path_to_class_names)))
  
  with open(os.path.join("/content","labels.names"),"wt") as file:
    with open(path_to_class_names,"r") as file2:
      for i in file2:
        file.write(i)
  
  data_path=os.path.join("/content","data.data")
  with open(data_path,"wt") as data:
    data.write("classes="+number_of_classes+"\n")
    data.write("train="+path_to_train_txt+"\n")
    data.write("valid="+path_to_valid_txt+"\n")
    data.write("names="+"/content/labels.names"+"\n")
  

train_path="/content/train.txt"
valid_path="/content/valid.txt"
labels="/content/drive/My Drive/Bath Thesis/classes.txt"

create_data(train_path,valid_path,labels)

for line in open("/content/data.data"):  
  print(line)

In [ ]:
#Copying custom config and train to train environment.
shutil.copy("/content/Computer-vision-and-Drones-Thesis/models/Yolo Config/yolov3-spp-custom.cfg","/content/yolov5/cfg")
shutil.copy("/content/Computer-vision-and-Drones-Thesis/models/Yolo Config/yolov3_train_custom.py","/content/yolov5")

In [ ]:
#By default pretrained --weights yolov3-spp-ultralytics.pt is used to initialize your model.

# In the weights, only one default weight is provided (transfer learning is used),although there are multiple models. 
#Thus, to be able to use other models, you either need to pretrain from scratch, or have the innitialization weights provided.
#"weights/yolov3-spp-ultralytics.pt is not compatible with cfg/yolov3-tiny.cfg. Specify --weights '' or specify a --cfg compatible with weights/yolov3-spp-ultralytics.pt.

In [ ]:
#Training.
#image size-multiple of 16.
#img size 352,the closest resolution to 360.
# when weights not '' uses availables pretrained weights.
#!python3 train.py --img 320 --cfg cfg/yolov3-tiny.cfg --data /content/data.data --epochs 5 --weights ''

import os
os.chdir("/content/yolov3")

!python3 train.py --img 352 --cfg cfg/yolov3-spp-custom.cfg --data /content/data.data --epochs 200  --name yolov3_results


In [ ]:
#Video object detection
import shutil
shutil.copy("/content/drive/My Drive/Bath Thesis/classes.txt","/content")
%cd /content/yolov3/

!python detect.py --cfg cfg/yolov3-spp-custom.cfg  --weights weights/best_yolov3_results.pt --source /content/high_quality.mp4 --names=/content/classes.txt --conf-thres 0.7 --img-size 1024